## Experimental Setup

In [1]:
%load_ext autoreload
%autoreload 2

import logging
from pathlib import Path
from typing import Callable

import numpy as np
import pandas as pd
import pyagrum as gum

from abapc import ABAPC
from utils.helpers import random_stability
from priors.generate_priors import PriorKnowledge, Constraints


def load_dataset(
    bif_path: Path, sample_size: int, seed: int
) -> tuple[gum.BayesNet, np.ndarray]:
    bn = gum.loadBN(str(bif_path))
    bn.name = bif_path.stem

    gum.initRandom(seed=seed)
    df = gum.generateSample(bn, sample_size, with_labels=False, random_order=False)[0]
    sorted_vars = sorted(df.columns)
    data = df[sorted_vars].to_numpy().astype(float)

    return bn, data

/vol/bitbucket/zl1724/.conda/new_aba/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Define Causal ABA Interface to call both implementations

In [2]:
from clingo import Function, Number

from utils.graph_utils import extract_test_elements_from_symbol


def CausalABA(
    n_nodes: int,
    compile_and_ground: Callable,
    facts_location: str = "",
    print_models: bool = True,
    skeleton_rules_reduction: bool = False,
    weak_constraints: bool = False,
    opt_mode: str = "optN",
    show: list = ["arrow"],
    pre_grounding: bool = False,
    disable_reground: bool = False,
    prior_knowledge: PriorKnowledge | None = None,
) -> list:
    """
    CausalABA, a function that takes in the number of nodes in a graph and a string of facts and returns a list of compatible causal graphs.

    """
    # (X, Y) -> their condition sets S
    indep_facts: dict[tuple, set[tuple]] = {}
    dep_facts: dict[tuple, set[tuple]] = {}
    facts = []
    ext_flag = False
    if facts_location:
        facts_loc = (
            facts_location.replace(".lp", "_I.lp")
            if weak_constraints
            else facts_location
        )
        logging.debug(f"   Loading facts from {facts_location}")
        with open(facts_loc, "r") as file:
            for line in file:
                if "dep" not in line or line.startswith("%"):
                    continue
                line_clean = line.replace("#external ", "").replace("\n", "")
                if "ext_" in line_clean:
                    ext_flag = True
                if weak_constraints:
                    statement, Is = line_clean.split(" I=")
                    I, truth = Is.split(",")
                    X, S, Y, dep_type = extract_test_elements_from_symbol(statement)
                    facts.append((X, S, Y, dep_type, statement, float(I), truth))
                else:
                    X, S, Y, dep_type = extract_test_elements_from_symbol(line_clean)
                    facts.append((X, S, Y, dep_type, line_clean, np.nan, "unknown"))

                assert (X not in S) and (Y not in S), f"X or Y in S: {line_clean}"
                condition_set = tuple(S)

                facts_group = indep_facts if "indep" in line_clean else dep_facts
                if (X, Y) not in facts_group:
                    facts_group[(X, Y)] = set()
                assert condition_set not in facts_group[(X, Y)], (
                    f"Redundant external fact: {line_clean}"
                )
                facts_group[(X, Y)].add(condition_set)

    ctl = compile_and_ground(
        n_nodes,
        facts_location,
        skeleton_rules_reduction,
        weak_constraints,
        indep_facts,
        dep_facts,
        opt_mode,
        show,
        pre_grounding,
        ext_flag,
        prior_knowledge,
    )

    facts = sorted(facts, key=lambda x: x[5], reverse=True)
    for fact in facts:
        ctl.assign_external(
            Function(
                fact[3],
                [
                    Number(fact[0]),
                    Number(fact[2]),
                    Function(fact[4].replace(").", "").split(",")[-1]),
                ],
            ),
            True,
        )
        logging.debug(f"   True fact: {fact[4]} I={fact[5]}, truth={fact[6]}")
    models = []
    logging.info("   Solving...")
    with ctl.solve(yield_=True) as handle:
        for model in handle:
            models.append(model.symbols(shown=True))
            if print_models:
                logging.info(f"Answer {len(models)}: {model}")
    n_models = int(ctl.statistics["summary"]["models"]["enumerated"])
    logging.info(f"Number of models: {n_models}")
    times = {
        key: ctl.statistics["summary"]["times"][key]
        for key in ["total", "cpu", "solve"]
    }
    logging.info(f"Times: {times}")
    remove_n = 0
    logging.info(f"Number of facts removed: {remove_n}")

    ## start removing facts if no models are found
    while n_models == 0 and remove_n < len(facts):
        remove_n += 1
        logging.info(f"Number of facts removed: {remove_n}")

        reground = False
        fact_to_remove = facts[-remove_n]
        X, S, Y, dep_type, fact_str = fact_to_remove[:5]
        logging.debug(f"Removing fact {fact_str}")

        facts_group = indep_facts if dep_type == "ext_indep" else dep_facts
        facts_group[(X, Y)].remove(tuple(S))
        if not facts_group[(X, Y)]:
            del facts_group[(X, Y)]
            reground = (
                disable_reground is False
                and skeleton_rules_reduction
                and (ext_flag is False or dep_type == "ext_indep")
            )
        else:
            logging.debug(
                f"   Not removing fact {fact_str} because there are multiple facts with the same X and Y"
            )
        ctl.assign_external(
            Function(
                dep_type,
                [
                    Number(X),
                    Number(Y),
                    Function(fact_str.replace(").", "").split(",")[-1]),
                ],
            ),
            None,
        )

        if reground:
            ### Save external statements
            logging.info("Recompiling and regrounding...")
            ctl = compile_and_ground(
                n_nodes,
                facts_location,
                skeleton_rules_reduction,
                weak_constraints,
                indep_facts,
                dep_facts,
                opt_mode,
                show,
                pre_grounding=pre_grounding,
                ext_flag=ext_flag,
                prior_knowledge=prior_knowledge,
            )
            for fact in facts[:-remove_n]:
                ctl.assign_external(
                    Function(
                        fact[3],
                        [
                            Number(fact[0]),
                            Number(fact[2]),
                            Function(fact[4].replace(").", "").split(",")[-1]),
                        ],
                    ),
                    True,
                )
                logging.debug(f"   True fact: {fact[4]} I={fact[5]}, truth={fact[6]}")
            for fact in facts[-remove_n:]:
                ctl.assign_external(
                    Function(
                        fact[3],
                        [
                            Number(fact[0]),
                            Number(fact[2]),
                            Function(fact[4].replace(").", "").split(",")[-1]),
                        ],
                    ),
                    None,
                )
                logging.debug(f"   False fact: {fact[4]} I={fact[5]}, truth={fact[6]}")
        models = []
        logging.info("   Solving...")
        with ctl.solve(yield_=True) as handle:
            for model in handle:
                models.append(model.symbols(shown=True))
                if print_models:
                    logging.info(f"Answer {len(models)}: {model}")
        n_models = int(ctl.statistics["summary"]["models"]["enumerated"])
        logging.info(f"Number of models: {n_models}")
        times = {
            key: ctl.statistics["summary"]["times"][key]
            for key in ["total", "cpu", "solve"]
        }
        logging.info(f"Times: {times}")

    return {
        "remove_n": remove_n,
        "statistics": ctl.statistics,
        "models": models,
    }


### Define the evaluation function to compare performance of two different implementations

In [3]:
import timeit
from typing import Any

from tqdm import tqdm

import abapc
import causalaba
from causalaba import compile_and_ground

abapc.tqdm = tqdm  # Avoid using progress bar widgets
causalaba.tqdm = tqdm  # Avoid using progress bar widgets


def compare_performance(
    bif_path: str | Path,
    sample_size: int,
    impl1: Callable[[gum.BayesNet, str, list], Callable],
    impl2: Callable[[gum.BayesNet, str, list], Callable],
    repeats: int = 1,
) -> tuple[str, int, float, float, list, list, np.ndarray, Any]:
    random_stability(2024)
    seeds = np.random.randint(0, 10000, size=repeats).tolist()

    if isinstance(bif_path, str):
        bif_path = Path(bif_path)

    impl1_return = []
    impl2_return = []

    for seed in seeds:
        bn_true, data = load_dataset(bif_path, sample_size=sample_size, seed=seed)
        facts_I_path, sepsets = ABAPC(
            data,
            seed=seed,
            alpha=0.01,
            indep_test="gsq",
            S_weight=False,
            out_mode="facts_only",
            scenario="prior_comparison",
        )
        facts_path = facts_I_path.replace("_I.lp", ".lp")


        impl1_return[-1]["time"] = timeit.timeit(impl1(bn_true, facts_path, impl1_return), number=1)
        impl2_return[-1]["time"] = timeit.timeit(impl2(bn_true, facts_path, impl2_return), number=1)

    return bif_path.stem, impl1_return, impl2_return, seeds, sepsets

## Compare Causal ABA performance with/without prior knowledge

In [4]:
def with_optional_prior(prior_df: pd.DataFrame | None = None):
    def impl_wrapper(bn: gum.BayesNet, facts_path: str, return_ref: list) -> Callable:
        prior_knowledge = None
        if prior_df is not None:
            filename = bn.property("name")
            prior_knowledge = PriorKnowledge(
                variables=bn.names(),
                constraints=Constraints(
                    forbidden=prior_df[prior_df["filename"] == filename]["priors"].iloc[
                        0
                    ],
                ),
            )

        def helper():
            return_ref.append(
                CausalABA(
                    n_nodes=bn.size(),
                    compile_and_ground=compile_and_ground,
                    facts_location=facts_path,
                    print_models=False,
                    skeleton_rules_reduction=True,
                    weak_constraints=True,
                    opt_mode="optN",
                    pre_grounding=True,  # Speed up solving by pre-grounding
                    prior_knowledge=prior_knowledge,
                )
            )

        return helper

    return impl_wrapper

In [5]:
from typing import Iterable

import abapc
from utils.graph_utils import DAGMetrics, dag2cpdag


repeats = 50
MAX_NODES = 9  # Skip graphs larger than this for performance reasons
report_cols = [
    "time",
    "remove_n",
    "cpdag_F1",
    "cpdag_shd",
    "cpdag_sid_low",
    "cpdag_sid_high",
]
report_stats = {col: ["mean", "std"] for col in report_cols}


def show_report(df: pd.DataFrame):
    groups = df.groupby(["dataset", "impl"])
    stats = groups.agg(report_stats)
    meta = groups.agg(
        {
            "num_nodes": "first",
            "num_edges": "first",
        }
    )
    meta["repeats"] = groups.size()
    meta.columns = pd.MultiIndex.from_product([["meta"], meta.columns])
    index = (
        groups["num_nodes"]
        .first()
        .reset_index()
        .sort_values(["num_nodes", "dataset", "impl"])
        .set_index(["dataset", "impl"])
        .index
    )

    return pd.concat([meta, stats], axis=1).loc[index]


def get_sorted_adjacency(bn: gum.BayesNet) -> np.ndarray:
    """Get the adjacency matrix of the Bayesian network sorted by variable names."""
    alphabetical_order = [bn.idFromName(var) for var in sorted(bn.names())]
    adj = bn.adjacencyMatrix()
    return adj[np.ix_(alphabetical_order, alphabetical_order)]


def run_experiment(prior_df: pd.DataFrame | None, datasets: Iterable[Path]):
    all_runs = []

    for random_graph_path in datasets:
        bn = gum.loadBN(str(random_graph_path))
        B_true = get_sorted_adjacency(bn)
        if bn.size() > MAX_NODES:
            continue

        print(f"Processing {random_graph_path.name}...")
        (
            bif_name,
            impl1_return,
            impl2_return,
            seeds,
            sepsets,
        ) = compare_performance(
            bif_path=random_graph_path,
            sample_size=5000,
            impl1=with_optional_prior(prior_df=prior_df),
            impl2=with_optional_prior(prior_df=None),
            repeats=repeats,
        )

        # Process both implementations in the same way
        for impl_name, results in [("org", impl2_return), ("new", impl1_return)]:
            for run_id, run in enumerate(results):
                # Mock ABAPC return
                abapc.CausalABA = lambda *args, **kwargs: (run["models"], False)
                best_model = abapc.ABAPC(
                    np.zeros((1, bn.size())),
                    seed=seeds[run_id],
                    sepsets=sepsets,
                    alpha=0.01,
                    indep_test="gsq",
                    S_weight=False,
                    out_mode="opt",
                    scenario="prior_comparison",
                )

                # DAG metrics
                B_est_dag = (best_model > 0).astype(int)
                mt_dag = DAGMetrics(B_est_dag, B_true).metrics

                # CPDAG metrics
                B_est_cpdag = (best_model != 0).astype(int)
                mt_cpdag = DAGMetrics(dag2cpdag(B_est_cpdag), B_true).metrics
                cpdag_sid = mt_cpdag.pop("sid")
                if not isinstance(cpdag_sid, tuple):
                    cpdag_sid = (cpdag_sid, cpdag_sid)
                mt_cpdag["sid_low"], mt_cpdag["sid_high"] = cpdag_sid

                # Save this run's results in a flat dict
                all_runs.append(
                    {
                        "dataset": bif_name,
                        "num_nodes": bn.size(),
                        "num_edges": bn.sizeArcs(),
                        "impl": impl_name,
                        "run_id": run_id,
                        "time": run["time"],
                        "remove_n": run["remove_n"],
                        **{f"dag_{k}": v for k, v in mt_dag.items()},
                        **{f"cpdag_{k}": v for k, v in mt_cpdag.items()},
                    }
                )

    return pd.DataFrame(all_runs)

### Bnlearn small datasets

In [6]:
bnlearn_small_datasets = list(Path("priors/datasets/bnlearn/").glob("*.bifxml"))
print("bnlearn_small_datasets:", bnlearn_small_datasets)
bnlearn_prior_df = pd.read_json("priors/datasets/bnlearn/prior_df.json")
bnlearn_prior_df

bnlearn_small_datasets: [PosixPath('priors/datasets/bnlearn/survey.bifxml'), PosixPath('priors/datasets/bnlearn/cancer.bifxml'), PosixPath('priors/datasets/bnlearn/sachs.bifxml'), PosixPath('priors/datasets/bnlearn/asia.bifxml'), PosixPath('priors/datasets/bnlearn/earthquake.bifxml')]


,filename,title,num_nodes,num_edges,variable_descriptions,priors,TP,FP,TN,FN,Precision,Recall,F-beta,run_id
0,asia,asia,8,8,{'smoke': 'smoking: whether or not the patient...,"[[bronc, tub], [xray, bronc], [smoke, asia], [...",18,0,8,2,1.000000,0.900000,0.995745,0
1,cancer,cancer,5,4,{'Cancer': 'Cancer: Indicates whether the pers...,"[[Xray, Dyspnoea], [Dyspnoea, Smoker], [Pollut...",5,0,4,1,1.000000,0.833333,0.992366,2
2,earthquake,earthquake,5,4,{'Earthquake': 'The occurrence of seismic acti...,"[[Earthquake, JohnCalls], [Burglary, JohnCalls...",5,0,4,1,1.000000,0.833333,0.992366,0
3,sachs,sachs,11,17,{'PKA': 'PKA (Protein Kinase A): cAMP-dependen...,"[[Jnk, Akt], [PKA, P38], [Akt, P38], [Plcg, P3...",11,2,15,27,0.846154,0.289474,0.787879,1
4,survey,survey,6,6,"{'A': 'Age: the age, recorded as young (young)...","[[A, S], [A, R], [A, O]]",3,0,6,6,1.000000,0.333333,0.928571,3


In [7]:
%%capture
res_df_bnlearn = run_experiment(
    prior_df=bnlearn_prior_df, datasets=bnlearn_small_datasets
)
res_df_bnlearn.to_csv(
    "results/2025/prior_integration_bnlearn_small.csv", index=False
)

<block>:1:39-51: info: atom does not occur in any rule head:
  ap(1,4,#P0,s0)

<block>:1:43-57: info: atom does not occur in any rule head:
  ap(1,4,#P0,s0y2)

<block>:1:45-60: info: atom does not occur in any rule head:
  ap(1,4,#P0,empty)



<block>:1:39-51: info: atom does not occur in any rule head:
  ap(1,4,#P0,s0)

<block>:1:43-57: info: atom does not occur in any rule head:
  ap(1,4,#P0,s0y2)

<block>:1:45-60: info: atom does not occur in any rule head:
  ap(1,4,#P0,empty)



<block>:1:39-51: info: atom does not occur in any rule head:
  ap(1,3,#P0,s2)

<block>:1:45-60: info: atom does not occur in any rule head:
  ap(1,4,#P0,empty)

<block>:1:39-51: info: atom does not occur in any rule head:
  ap(1,4,#P0,s0)

<block>:1:45-60: info: atom does not occur in any rule head:
  ap(1,4,#P0,empty)

<block>:1:39-51: info: atom does not occur in any rule head:
  ap(1,4,#P0,s0)

<block>:1:45-60: info: atom does not occur in any rule head:
  ap(1,4,#P0,empty)



<block>:1:39-51: info: atom does not occur in any rule head:
  ap(1,3,#P0,s2)

<block>:1:45-60: info: atom does not occur in any rule head:
  ap(1,4,#P0,empty)

<block>:1:39-51: info: atom does not occur in any rule head:
  ap(1,4,#P0,s0)

<block>:1:45-60: info: atom does not occur in any rule head:
  ap(1,4,#P0,empty)

<block>:1:39-51: info: atom does not occur in any rule head:
  ap(1,4,#P0,s0)

<block>:1:45-60: info: atom does not occur in any rule head:
  ap(1,4,#P0,empty)



<block>:1:39-51: info: atom does not occur in any rule head:
  ap(1,4,#P0,s0)

<block>:1:45-60: info: atom does not occur in any rule head:
  ap(1,4,#P0,empty)



...

<block>:1:39-51: info: atom does not occur in any rule head:
  ap(2,7,#P0,s5)

<block>:1:43-57: info: atom does not occur in any rule head:
  ap(3,7,#P0,s2y4)

<block>:1:45-60: info: atom does not occur in any rule head:
  ap(2,7,#P0,empty)

<block>:1:39-51: info: atom does not occur in any rule head:
  ap(2,7,#P0,s5)

<block>:1:43-57: info: atom does not occur in any rule head:
  ap(3,7,#P0,s2y4)

<block>:1:45-60: info: atom does not occur in any rule head:
  ap(2,7,#P0,empty)



<block>:1:39-51: info: atom does not occur in any rule head:
  ap(2,7,#P0,s5)

<block>:1:43-57: info: atom does not occur in any rule head:
  ap(3,7,#P0,s2y4)

<block>:1:45-60: info: atom does not occur in any rule head:
  ap(2,7,#P0,empty)



<block>:1:39-51: info: atom does not occur in any rule head:
  ap(2,7,#P0,s5)

<block>:1:43-57: info: atom does not occur in any rule head:
  ap(3,7,#P0,s2y4)

<block>:1:45-60: info: atom does not occur in any rule head:
  ap(2,7,#P0,empty)

<block>:1:39-51: info: atom does not occur in any rule head:
  ap(2,7,#P0,s5)

<block>:1:43-57: info: atom does not occur in any rule head:
  ap(3,7,#P0,s2y4)

<block>:1:45-60: info: atom does not occur in any rule head:
  ap(2,7,#P0,empty)

<block>:1:39-51: info: atom does not occur in any rule head:
  ap(2,7,#P0,s5)

<block>:1:43-57: info: atom does not occur in any rule head:
  ap(3,7,#P0,s2y6)

<block>:1:45-60: info: atom does not occur in any rule head:
  ap(2,7,#P0,empty)



[1] "The estimated graph is not chordal, i.e. it is not a CPDAG! We thus consider local expansions of the graph (some combinations of which may lead to cycles)."


<block>:1:39-51: info: atom does not occur in any rule head:
  ap(3,4,#P0,s2)

<block>:1:43-57: info: atom does not occur in any rule head:
  ap(0,4,#P0,s2y3)

<block>:1:45-60: info: atom does not occur in any rule head:
  ap(3,4,#P0,empty)

<block>:1:39-51: info: atom does not occur in any rule head:
  ap(3,4,#P0,s2)

<block>:1:43-57: info: atom does not occur in any rule head:
  ap(0,4,#P0,s2y3)

<block>:1:45-60: info: atom does not occur in any rule head:
  ap(3,4,#P0,empty)



In [8]:
bnlearn_report = show_report(res_df_bnlearn)
bnlearn_report.to_csv("results/2025/prior_integration_bnlearn_small_report.csv")
# To load the view, use
# bnlearn_report = pd.read_csv(
#     "results/2025/prior_integration_bnlearn_small_report.csv",
#     index_col=[0, 1],
#     header=[0, 1],
# )
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(bnlearn_report)

meta                         time            remove_n  \
                num_nodes num_edges repeats       mean        std     mean   
dataset    impl                                                              
cancer     new          5         4      50   0.436892   0.216604     7.80   
           org          5         4      50   0.506319   0.281763     7.74   
earthquake new          5         4      50   0.029978   0.026937     0.70   
           org          5         4      50   0.025487   0.014106     0.36   
survey     new          6         6      50   2.128081   0.829938    26.04   
           org          6         6      50   3.505351   1.901849    25.94   
asia       new          8         8      50   8.194210   1.773714   114.36   
           org          8         8      50  45.939552  27.681830    33.56   

                           cpdag_F1           cpdag_shd            \
                      std      mean       std      mean       std   
dataset    impl                                                     
cancer     new   2.373321  0.649915  0.189144      2.08  1.454620   
           org   2.405860  0.627350  0.173317      2.30  1.528638   
earthquake new   2.052873  0.988888  0.078574      0.06  0.424264   
           org   1.045105  0.977334  0.091623      0.18  0.660550   
survey     new   8.785331  0.584302  0.117093      4.66  1.745081   
           org   8.828086  0.537174  0.138345      4.96  1.511318   
asia       new   9.901062  0.599618  0.138221      2.90  2.101991   
           org   5.686217  0.329544  0.080145     10.22  2.772238   

                cpdag_sid_low           cpdag_sid_high            
                         mean       std           mean       std  
dataset    impl                                                   
cancer     new           6.02  3.883455          11.62  3.885557  
           org           6.22  4.057193          11.64  3.544239  
earthquake new           0.00  0.000000           5.18  1.272792  
           org           0.00  0.000000           5.68  2.235520  
survey     new          12.08  3.885294          17.90  5.496752  
           org          12.56  3.812761          18.24  4.684146  
asia       new           6.98  6.867522          25.08  8.616169  
           org          12.62  5.753047          25.70  8.459387

### Synthetic datasets

In [9]:
synthetic_datasets = list(Path("priors/datasets/random_graphs/heuristic_by_semantic/").glob("*.bifxml"))
print("synthetic_datasets:", synthetic_datasets)
synthetic_prior_df = pd.read_json("priors/datasets/random_graphs/prior_df.json")
synthetic_prior_df

synthetic_datasets: [PosixPath('priors/datasets/random_graphs/heuristic_by_semantic/factors_influencing_well_being_and_resources.bifxml'), PosixPath('priors/datasets/random_graphs/heuristic_by_semantic/pharmacological_and_cardiac_health_dynamics.bifxml'), PosixPath('priors/datasets/random_graphs/heuristic_by_semantic/interacting_health_conditions_and_outcomes.bifxml'), PosixPath('priors/datasets/random_graphs/heuristic_by_semantic/clinical_variables_in_cardiovascular_and_gastrointestinal_health.bifxml'), PosixPath('priors/datasets/random_graphs/heuristic_by_semantic/clinical_pathways_in_cardiac_and_abdominal_health.bifxml'), PosixPath('priors/datasets/random_graphs/heuristic_by_semantic/interconnected_health_and_socio_economic_outcomes.bifxml'), PosixPath('priors/datasets/random_graphs/heuristic_by_semantic/medical_conditions_and_outcomes.bifxml')]


,filename,title,num_nodes,num_edges,variable_descriptions,priors,TP,FP,TN,FN,Precision,Recall,F-beta,run_id
0,clinical_pathways_in_cardiac_and_abdominal_health,clinical_pathways_in_cardiac_and_abdominal_health,7,7,{'shivering': 'An involuntary muscular contrac...,"[[shivering, ascites], [shivering, hospital_ad...",10,0,7,4,1,0.714286,0.984848,4
1,clinical_variables_in_cardiovascular_and_gastr...,clinical_variables_in_cardiovascular_and_gastr...,5,4,{'rectal_varices': 'Abnormally enlarged and to...,"[[heart_failure, lower_gastrointestinal_bleedi...",4,0,4,2,1,0.666667,0.981132,9
2,factors_influencing_well_being_and_resources,factors_influencing_well_being_and_resources,9,8,"{'property_damage': 'The impairment, destructi...","[[early_death, separation], [heart_failure, di...",9,0,8,19,1,0.321429,0.924901,7
3,interacting_health_conditions_and_outcomes,interacting_health_conditions_and_outcomes,11,10,{'kidney_failure': 'A medical condition where ...,"[[heart_trouble, osteoporosis_associated_fract...",14,0,10,31,1,0.311111,0.921519,1
4,interconnected_health_and_socio_economic_outcomes,interconnected_health_and_socio_economic_outcomes,10,11,{'early_death': 'The occurrence of death at an...,"[[stagnant_economy, hospitalization], [clostri...",12,0,11,22,1,0.352941,0.934132,3
5,medical_conditions_and_outcomes,medical_conditions_and_outcomes,8,7,{'early_death': 'The occurrence of mortality a...,"[[coronary_heart_disease, imbalanced_ph], [lam...",10,0,7,11,1,0.476190,0.959410,9
6,pharmacological_and_cardiac_health_dynamics,pharmacological_and_cardiac_health_dynamics,6,6,{'palpitations': 'The subjective experience of...,"[[palpitations, appetite_suppression], [heart_...",7,0,6,2,1,0.777778,0.989130,1


In [10]:
%%capture
synthetic_res_df = run_experiment(
    prior_df=synthetic_prior_df, datasets=synthetic_datasets
)
synthetic_res_df.to_csv(
    "results/2025/prior_integration_synthetic.csv", index=False
)

<block>:1:18-37: info: atom does not occur in any rule head:
  ext_indep(5,6,s8y1)

<block>:1:20-37: info: atom does not occur in any rule head:
  ext_dep(5,6,s8y3)

<block>:1:22-41: info: atom does not occur in any rule head:
  ext_dep(5,6,s8y1y3)

<block>:1:18-37: info: atom does not occur in any rule head:
  ext_indep(5,6,s8y1)

<block>:1:20-37: info: atom does not occur in any rule head:
  ext_dep(5,6,s8y3)

<block>:1:22-41: info: atom does not occur in any rule head:
  ext_dep(5,6,s8y1y3)

<block>:1:18-37: info: atom does not occur in any rule head:
  ext_indep(5,6,s8y1)

<block>:1:20-37: info: atom does not occur in any rule head:
  ext_dep(3,6,s8y1)

<block>:1:22-41: info: atom does not occur in any rule head:
  ext_dep(5,6,s8y1y3)



<block>:1:18-37: info: atom does not occur in any rule head:
  ext_indep(5,6,s8y1)

<block>:1:20-37: info: atom does not occur in any rule head:
  ext_dep(3,6,s8y1)

<block>:1:22-41: info: atom does not occur in any rule head:
  ext_dep(5,6,s8y1y3)

<block>:1:18-37: info: atom does not occur in any rule head:
  ext_indep(5,6,s8y1)

<block>:1:20-37: info: atom does not occur in any rule head:
  ext_dep(3,6,s8y1)

<block>:1:22-41: info: atom does not occur in any rule head:
  ext_dep(5,6,s8y1y3)

<block>:1:18-37: info: atom does not occur in any rule head:
  ext_indep(5,6,s8y1)

<block>:1:20-37: info: atom does not occur in any rule head:
  ext_dep(3,6,s8y1)

<block>:1:22-41: info: atom does not occur in any rule head:
  ext_dep(5,6,s8y1y3)



<block>:1:18-37: info: atom does not occur in any rule head:
  ext_indep(5,6,s8y1)

<block>:1:20-37: info: atom does not occur in any rule head:
  ext_dep(1,6,s8y3)

<block>:1:22-41: info: atom does not occur in any rule head:
  ext_dep(5,6,s8y1y3)

<block>:1:18-37: info: atom does not occur in any rule head:
  ext_indep(5,6,s8y1)

<block>:1:20-37: info: atom does not occur in any rule head:
  ext_dep(3,6,s8y1)

<block>:1:22-41: info: atom does not occur in any rule head:
  ext_dep(5,6,s8y1y3)



<block>:1:18-37: info: atom does not occur in any rule head:
  ext_indep(5,6,s8y1)

<block>:1:20-37: info: atom does not occur in any rule head:
  ext_dep(1,6,s8y3)

<block>:1:22-41: info: atom does not occur in any rule head:
  ext_dep(5,6,s8y1y3)

<block>:1:18-37: info: atom does not occur in any rule head:
  ext_indep(5,6,s8y1)

<block>:1:20-37: info: atom does not occur in any rule head:
  ext_dep(5,6,s8y3)

<block>:1:22-41: info: atom does not occur in any rule head:
  ext_dep(5,6,s8y1y3)



<block>:1:18-37: info: atom does not occur in any rule head:
  ext_indep(5,6,s8y1)

<block>:1:20-37: info: atom does not occur in any rule head:
  ext_dep(3,6,s8y1)

<block>:1:22-41: info: atom does not occur in any rule head:
  ext_dep(5,6,s8y1y3)

<block>:1:18-37: info: atom does not occur in any rule head:
  ext_indep(5,6,s8y1)

<block>:1:20-37: info: atom does not occur in any rule head:
  ext_dep(1,6,s8y3)

<block>:1:22-41: info: atom does not occur in any rule head:
  ext_dep(5,6,s8y1y3)



...

<block>:1:39-51: info: atom does not occur in any rule head:
  ap(1,3,#P0,s5)

<block>:1:45-60: info: atom does not occur in any rule head:
  ap(1,3,#P0,empty)



<block>:1:39-51: info: atom does not occur in any rule head:
  ap(4,6,#P0,s2)

<block>:1:45-60: info: atom does not occur in any rule head:
  ap(4,6,#P0,empty)



<block>:1:39-51: info: atom does not occur in any rule head:
  ap(4,6,#P0,s2)

<block>:1:45-60: info: atom does not occur in any rule head:
  ap(4,6,#P0,empty)



<block>:1:39-51: info: atom does not occur in any rule head:
  ap(0,1,#P0,s6)

<block>:1:45-60: info: atom does not occur in any rule head:
  ap(0,1,#P0,empty)



<block>:1:39-51: info: atom does not occur in any rule head:
  ap(0,1,#P0,s6)

<block>:1:45-60: info: atom does not occur in any rule head:
  ap(0,1,#P0,empty)



In [11]:
synthetic_report = show_report(synthetic_res_df)
synthetic_report.to_csv("results/2025/prior_integration_synthetic_report.csv")
# To load the view, use
# synthetic_report = pd.read_csv(
#     "results/2025/prior_integration_synthetic_report.csv",
#     index_col=[0, 1],
#     header=[0, 1],
# )
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(synthetic_report)

meta            \
                                                        num_nodes num_edges   
dataset                                            impl                       
clinical_variables_in_cardiovascular_and_gastro... new          5         4   
                                                   org          5         4   
pharmacological_and_cardiac_health_dynamics        new          6         6   
                                                   org          6         6   
clinical_pathways_in_cardiac_and_abdominal_health  new          7         7   
                                                   org          7         7   
medical_conditions_and_outcomes                    new          8         7   
                                                   org          8         7   
factors_influencing_well_being_and_resources       new          9         8   
                                                   org          9         8   

                                                                       time  \
                                                        repeats        mean   
dataset                                            impl                       
clinical_variables_in_cardiovascular_and_gastro... new       50    0.085994   
                                                   org       50    0.079141   
pharmacological_and_cardiac_health_dynamics        new       50    0.094437   
                                                   org       50    0.116436   
clinical_pathways_in_cardiac_and_abdominal_health  new       50    1.458400   
                                                   org       50    2.594479   
medical_conditions_and_outcomes                    new       50    1.105311   
                                                   org       50    2.732736   
factors_influencing_well_being_and_resources       new       50    9.930081   
                                                   org       50  109.415107   

                                                                    remove_n  \
                                                                std     mean   
dataset                                            impl                        
clinical_variables_in_cardiovascular_and_gastro... new     0.036612    10.34   
                                                   org     0.029361     9.78   
pharmacological_and_cardiac_health_dynamics        new     0.083969     6.50   
                                                   org     0.119712     6.26   
clinical_pathways_in_cardiac_and_abdominal_health  new     0.703391    31.08   
                                                   org     1.481721    30.54   
medical_conditions_and_outcomes                    new     2.001451    10.86   
                                                   org     5.299585    10.50   
factors_influencing_well_being_and_resources       new    21.053001    17.78   
                                                   org   315.931122    17.74   

                                                                    cpdag_F1  \
                                                               std      mean   
dataset                                            impl                        
clinical_variables_in_cardiovascular_and_gastro... new    3.788328  0.628332   
                                                   org    3.059212  0.549415   
pharmacological_and_cardiac_health_dynamics        new    6.996355  0.244017   
                                                   org    6.381766  0.245108   
clinical_pathways_in_cardiac_and_abdominal_health  new   10.651895  0.491781   
                                                   org   10.363220  0.471443   
medical_conditions_and_outcomes                    new   10.546322  0.725696   
                                                   org   10.170486  0.711450   
factors_influencing_well_being_and_resources       new   14.188656  0.625020   
    